# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,sarangani,5.4033,125.4636,27.35,77,23,9.28,PH,1712791178
1,1,san antonio de pale,-1.4014,5.6325,28.29,81,82,4.54,GQ,1712791096
2,2,sitka,57.0531,-135.3300,6.94,70,100,5.14,US,1712791046
3,3,ajdabiya,30.7554,20.2263,15.86,62,0,4.00,LY,1712791179
4,4,adamstown,-25.0660,-130.1015,22.67,85,100,6.84,PN,1712791006


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 500,
    frame_height = 300,
    size="Humidity",
    color="City"
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_filter_df = city_data_df.loc[(city_data_df["Max Temp"]<27) & (city_data_df["Max Temp"]>21) &
                                      (city_data_df["Cloudiness"]==0) & (city_data_df["Wind Speed"]<4.5)&
                                      (city_data_df["Humidity"]>60)]

# Drop any rows with null values
city_data_filter_df.dropna()

# Display sample data
city_data_filter_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
232,232,saint-pierre,-21.3393,55.4781,23.82,73,0,3.09,RE,1712791104
276,276,arraial do cabo,-22.9661,-42.0278,25.40,87,0,4.41,BR,1712790953
460,460,sao joao da barra,-21.6403,-41.0511,25.05,86,0,3.94,BR,1712791158
520,520,pacific grove,36.6177,-121.9166,25.36,73,0,3.58,US,1712791004
529,529,morehead city,34.7229,-76.7260,22.01,74,0,3.09,US,1712791035


### Step 3: Create a new DataFrame called `hotel_df`.

In [8]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_filter_df[["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
232,saint-pierre,RE,-21.3393,55.4781,73,
276,arraial do cabo,BR,-22.9661,-42.0278,87,
460,sao joao da barra,BR,-21.6403,-41.0511,86,
520,pacific grove,US,36.6177,-121.9166,73,
529,morehead city,US,34.7229,-76.7260,74,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [9]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "limit":1,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{row["Lng"]},{row["Lat"]},{radius}'
    params["bias"] = f'proximity:{row["Lng"]},{row["Lat"]}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
saint-pierre - nearest hotel: Tropic Hotel
arraial do cabo - nearest hotel: No hotel found
sao joao da barra - nearest hotel: Pousada Mediterrâneo
pacific grove - nearest hotel: Pacific Grove Inn
morehead city - nearest hotel: Bask Hotel Big Rock Landing, Trademark Collection by Wyndham


,City,Country,Lat,Lng,Humidity,Hotel Name
232,saint-pierre,RE,-21.3393,55.4781,73,Tropic Hotel
276,arraial do cabo,BR,-22.9661,-42.0278,87,No hotel found
460,sao joao da barra,BR,-21.6403,-41.0511,86,Pousada Mediterrâneo
520,pacific grove,US,36.6177,-121.9166,73,Pacific Grove Inn
529,morehead city,US,34.7229,-76.7260,74,"Bask Hotel Big Rock Landing, Trademark Collect..."


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size="Humidity",
    color="City",
    hover_cols=["Hotel Name","Country"]
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)